In [2]:
import numpy as np
import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import Dataset
import pickle
from utils.helper import read_py150k_code, read_file_to_string
import regex as re

fname_prefix = "/data/users/team2_capstone/code-style-probing/"

In [3]:
docstring_df = pd.read_csv(fname_prefix + "data/labeled_code/bq_updated_docstring_outlier.csv")
docstring_df = docstring_df[docstring_df['uncommented_content'].notnull()]
docstring_df = docstring_df[docstring_df['no_docstring_content'].notnull()]

In [4]:
display(docstring_df)

,Unnamed: 0.1,Unnamed: 0,repository,filepath,forks,issue_events,stars,parse_error,line_count,comment_count,...,upper_camel_case_method,other_case_method,upper_case_class,lower_camel_case_class,upper_case_method,file,labels,content,no_docstring_content,uncommented_content
0,0,0,jbkalmbach/kbmod,analysis/rawGenerated/batchGenerate.py,1,32,1,1,51.0,0.0,...,NaN,NaN,NaN,NaN,NaN,jbkalmbach/kbmodanalysis/rawGenerated/batchGen...,-1,from kbmodpy import kbmod as kb\nimport rawSea...,from kbmodpy import kbmod as kb\nimport rawSea...,from kbmodpy import kbmod as kb\nimport rawSea...
1,1,1,pymedusa/SickRage,medusa/providers/torrent/rss/rsstorrent.py,22,763,27,1,170.0,8.0,...,NaN,NaN,NaN,NaN,NaN,pymedusa/SickRagemedusa/providers/torrent/rss/...,-1,"# coding=utf-8\n\n""""""Provider code for RSS Tor...","""""""Provider code for RSS Torrents.""""""\nfrom __...","""""""Provider code for RSS Torrents.""""""\nfrom __..."
2,2,2,ipdata/python,ipdata/test_cli.py,6,15,16,1,135.0,34.0,...,NaN,NaN,NaN,NaN,NaN,ipdata/pythonipdata/test_cli.py,-1,import multiprocessing\nimport sys\nimport uni...,import multiprocessing\nimport sys\nimport uni...,import multiprocessing\nimport sys\nimport uni...
3,3,3,demisto/content,Packs/AnsibleLinux/Integrations/AnsibleACME/An...,1328,52,797,1,59.0,11.0,...,NaN,NaN,NaN,NaN,NaN,demisto/contentPacks/AnsibleLinux/Integrations...,-1,import traceback\nimport ssh_agent_setup\nimpo...,import traceback\nimport ssh_agent_setup\nimpo...,import traceback\nimport ssh_agent_setup\nimpo...
4,4,4,hidat/audio_pipeline,review_parser/mb_release.py,1,26,4,1,76.0,12.0,...,NaN,NaN,NaN,NaN,NaN,hidat/audio_pipelinereview_parser/mb_release.py,-1,import json\nimport os\nimport os.path as path...,import json\nimport os\nimport os.path as path...,import json\nimport os\nimport os.path as path...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875091,875091,87927,operepo/ope,libs/gluon/packages/dal/pydal/adapters/sqlite.py,8,138,9,1,279.0,14.0,...,NaN,10.0,NaN,NaN,2.0,operepo/opelibs/gluon/packages/dal/pydal/adapt...,-1,# -*- coding: utf-8 -*-\nimport copy\nimport d...,import copy\nimport datetime\nimport locale\ni...,import copy\nimport datetime\nimport locale\ni...
875092,875092,87928,DOV-Vlaanderen/pydov,setup.py,17,287,31,1,66.0,3.0,...,NaN,NaN,NaN,NaN,NaN,DOV-Vlaanderen/pydovsetup.py,-1,#!/usr/bin/env python\n# -*- coding: utf-8 -*-...,"""""""The setup script.""""""\nfrom setuptools impor...","""""""The setup script.""""""\nfrom setuptools impor..."
875093,875093,87929,slogan621/tscharts,apps/xrayuploader/xrayuploader.py,8,97,16,1,502.0,20.0,...,NaN,3.0,NaN,NaN,NaN,slogan621/tschartsapps/xrayuploader/xrayupload...,-1,#(C) Copyright Syd Logan 2020\n#(C) Copyright ...,"import getopt, sys\nimport json\nimport time\n...","import getopt, sys\nimport json\nimport time\n..."
875094,875094,87930,seasonstar/bibi,application/models/order/logistic.py,365,12,1016,1,434.0,1.0,...,NaN,NaN,NaN,NaN,NaN,seasonstar/bibiapplication/models/order/logist...,-1,# -*- coding: utf-8 -*-\nimport math\nimport r...,import math\nimport random\nimport datetime\ni...,import math\nimport random\nimport datetime\ni...


In [5]:
#split docstring_df for short data items and long data items
def len_content(source):
    return (len(source))
docstring_df['lens'] =  (docstring_df['uncommented_content'].apply(len_content))
short_df = (docstring_df[docstring_df['lens'] < 2000])
long_df = (docstring_df[docstring_df['lens'] >= 2000])
display(short_df)

,Unnamed: 0.1,Unnamed: 0,repository,filepath,forks,issue_events,stars,parse_error,line_count,comment_count,...,other_case_method,upper_case_class,lower_camel_case_class,upper_case_method,file,labels,content,no_docstring_content,uncommented_content,lens
0,0,0,jbkalmbach/kbmod,analysis/rawGenerated/batchGenerate.py,1,32,1,1,51.0,0.0,...,NaN,NaN,NaN,NaN,jbkalmbach/kbmodanalysis/rawGenerated/batchGen...,-1,from kbmodpy import kbmod as kb\nimport rawSea...,from kbmodpy import kbmod as kb\nimport rawSea...,from kbmodpy import kbmod as kb\nimport rawSea...,1124
3,3,3,demisto/content,Packs/AnsibleLinux/Integrations/AnsibleACME/An...,1328,52,797,1,59.0,11.0,...,NaN,NaN,NaN,NaN,demisto/contentPacks/AnsibleLinux/Integrations...,-1,import traceback\nimport ssh_agent_setup\nimpo...,import traceback\nimport ssh_agent_setup\nimpo...,import traceback\nimport ssh_agent_setup\nimpo...,1774
5,5,5,ucoin-io/ucoin-python-api,examples/request_web_socket_block.py,3,33,1,1,77.0,13.0,...,NaN,NaN,NaN,NaN,ucoin-io/ucoin-python-apiexamples/request_web_...,-1,"""""""\nCopyright 2014-2021 Vincent Texier <vit@...","""""""\nCopyright 2014-2021 Vincent Texier <vit@...","""""""\nCopyright 2014-2021 Vincent Texier <vit@...",1812
9,9,9,dkandalov/katas,python/async/6.py,3,2,7,1,54.0,2.0,...,NaN,NaN,NaN,NaN,dkandalov/kataspython/async/6.py,-1,import asyncio\n\nfrom common import expect_to...,import asyncio\nfrom common import expect_to_b...,import asyncio\nfrom common import expect_to_b...,609
12,12,12,CivicKnowledge/ambry,test/bundle_tests/ingest.example.com/headersty...,8,303,4,1,6.0,0.0,...,NaN,NaN,NaN,NaN,CivicKnowledge/ambrytest/bundle_tests/ingest.e...,20,import ambry.bundle \n\n\nclass Bundle(ambry.b...,import ambry.bundle\n\nclass Bundle(ambry.bund...,import ambry.bundle\n\nclass Bundle(ambry.bund...,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875083,875083,87919,Splawik/pytigon,pytigon/prj/schpytigondemo/tasks_demo/__init__.py,1,2,10,1,12.0,0.0,...,NaN,NaN,NaN,NaN,Splawik/pytigonpytigon/prj/schpytigondemo/task...,-1,from django.utils.translation import gettext_l...,from django.utils.translation import gettext_l...,from django.utils.translation import gettext_l...,299
875084,875084,87920,kytulendu/Gw2Browser,extern/glew/build/conan/test_package/conanfile.py,16,5,43,1,22.0,0.0,...,NaN,NaN,NaN,NaN,kytulendu/Gw2Browserextern/glew/build/conan/te...,20,"from conans import ConanFile, CMake\nimport os...","from conans import ConanFile, CMake\nimport os...","from conans import ConanFile, CMake\nimport os...",715
875087,875087,87923,martinstastny/django-simplestore,simplestore/products/migrations/0003_auto_2017...,11,1,36,1,25.0,2.0,...,NaN,NaN,NaN,NaN,martinstastny/django-simplestoresimplestore/pr...,20,# -*- coding: utf-8 -*-\n# Generated by Django...,from __future__ import unicode_literals\nfrom ...,from __future__ import unicode_literals\nfrom ...,399
875089,875089,87925,indico/indico-plugins,piwik/indico_piwik/forms.py,38,58,22,1,37.0,6.0,...,NaN,NaN,NaN,NaN,indico/indico-pluginspiwik/indico_piwik/forms.py,-1,# This file is part of the Indico plugins.\n# ...,"from wtforms import BooleanField, IntegerField...","from wtforms import BooleanField, IntegerField...",1467


In [6]:
#remove start docstrings from short_df 'no_docstring'
import ast
def undocstring(source): 
    if "\"\"\"" in source or "\'\'\'" in source :
        try:
            parsed = ast.parse(source)
            for node in ast.walk(parsed):
                #print("Node value is : ",node.body[0].value.s)
                        
                if not isinstance(node, (ast.Module, ast.FunctionDef, ast.ClassDef, ast.AsyncFunctionDef)):
                    continue

                if not len(node.body):
                    continue

                if not isinstance(node.body[0], ast.Expr):
                    continue

                if not hasattr(node.body[0], 'value') or not isinstance(node.body[0].value, ast.Str):
                    continue

                node.body = node.body[1:]
                new_code = ast.unparse(parsed)#toLower().visit(parsed))
                #print(new_code)
                return new_code
        except:
            return 'nan'
    return source 
repaired_df = short_df.query("no_docstring_content != uncommented_content")
#print (undocstring (repaired_df.iloc[0]['no_docstring_content']))
repaired_df['no_docstring_content'] = repaired_df['no_docstring_content'].apply(undocstring)
display(repaired_df)
repaired_df = repaired_df[repaired_df['no_docstring_content'].notnull()]
display(repaired_df)

/tmp/ipykernel_105163/1376333306.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repaired_df['no_docstring_content'] = repaired_df['no_docstring_content'].apply(undocstring)


,Unnamed: 0.1,Unnamed: 0,repository,filepath,forks,issue_events,stars,parse_error,line_count,comment_count,...,other_case_method,upper_case_class,lower_camel_case_class,upper_case_method,file,labels,content,no_docstring_content,uncommented_content,lens
3,3,3,demisto/content,Packs/AnsibleLinux/Integrations/AnsibleACME/An...,1328,52,797,1,59.0,11.0,...,NaN,NaN,NaN,NaN,demisto/contentPacks/AnsibleLinux/Integrations...,-1,import traceback\nimport ssh_agent_setup\nimpo...,import traceback\nimport ssh_agent_setup\nimpo...,import traceback\nimport ssh_agent_setup\nimpo...,1774
5,5,5,ucoin-io/ucoin-python-api,examples/request_web_socket_block.py,3,33,1,1,77.0,13.0,...,NaN,NaN,NaN,NaN,ucoin-io/ucoin-python-apiexamples/request_web_...,-1,"""""""\nCopyright 2014-2021 Vincent Texier <vit@...",import asyncio\nimport json\nfrom _socket impo...,"""""""\nCopyright 2014-2021 Vincent Texier <vit@...",1812
26,26,26,helena-project/beetle,controller/gatt/views.py,2,1,15,1,78.0,0.0,...,NaN,NaN,NaN,NaN,helena-project/beetlecontroller/gatt/views.py,-1,"\nfrom django.http import JsonResponse, HttpRe...","from django.http import JsonResponse, HttpResp...","from django.http import JsonResponse, HttpResp...",1852
29,29,29,python-security/pyt,examples/django.nV/taskManager/forms.py,275,166,2145,1,90.0,14.0,...,NaN,NaN,NaN,NaN,python-security/pytexamples/django.nV/taskMana...,-1,# _ _ __ __\n# ...,"from taskManager.models import Project, Task\n...",""""""" forms.py contains various Django forms for...",1959
36,36,36,ray-project/ray,python/ray/experimental/state/common.py,4242,17339,20545,1,94.0,2.0,...,NaN,NaN,NaN,NaN,ray-project/raypython/ray/experimental/state/c...,-1,import logging\n\nfrom dataclasses import data...,import logging\nfrom dataclasses import datacl...,import logging\nfrom dataclasses import datacl...,1643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875012,875012,87848,ManiacalLabs/BiblioPixel,bibliopixel/builder/pixels.py,72,967,249,1,70.0,0.0,...,NaN,NaN,NaN,NaN,ManiacalLabs/BiblioPixelbibliopixel/builder/pi...,-1,"import itertools, weakref\nfrom .. colors impo...","import itertools, weakref\nfrom ..colors impor...","import itertools, weakref\nfrom ..colors impor...",1899
875031,875031,87867,QQuick/Transcrypt,transcrypt/development/automated_tests/re/basi...,231,922,2175,1,38.0,4.0,...,NaN,NaN,NaN,NaN,QQuick/Transcrypttranscrypt/development/automa...,-1,from org.transcrypt.stubs.browser import __pra...,from org.transcrypt.stubs.browser import __pra...,from org.transcrypt.stubs.browser import __pra...,925
875042,875042,87878,softlayer/softlayer-python,SoftLayer/CLI/hardware/authorize_storage.py,180,1060,122,1,25.0,1.0,...,NaN,NaN,NaN,NaN,softlayer/softlayer-pythonSoftLayer/CLI/hardwa...,-1,"""""""Authorize File or Block Storage to a Hardwa...",import click\nimport SoftLayer\nfrom SoftLayer...,"""""""Authorize File or Block Storage to a Hardwa...",812
875071,875071,87907,clic-lab/blocks,BlockWorldRoboticAgent/model/mix_and_gen_q_val...,12,2,39,1,53.0,3.0,...,NaN,NaN,NaN,NaN,clic-lab/blocksBlockWorldRoboticAgent/model/mi...,-1,import tensorflow as tf\n\n\nclass MixAndGener...,import tensorflow as tf\n\nclass MixAndGenerat...,import tensorflow as tf\n\nclass MixAndGenerat...,1881


,Unnamed: 0.1,Unnamed: 0,repository,filepath,forks,issue_events,stars,parse_error,line_count,comment_count,...,other_case_method,upper_case_class,lower_camel_case_class,upper_case_method,file,labels,content,no_docstring_content,uncommented_content,lens
3,3,3,demisto/content,Packs/AnsibleLinux/Integrations/AnsibleACME/An...,1328,52,797,1,59.0,11.0,...,NaN,NaN,NaN,NaN,demisto/contentPacks/AnsibleLinux/Integrations...,-1,import traceback\nimport ssh_agent_setup\nimpo...,import traceback\nimport ssh_agent_setup\nimpo...,import traceback\nimport ssh_agent_setup\nimpo...,1774
5,5,5,ucoin-io/ucoin-python-api,examples/request_web_socket_block.py,3,33,1,1,77.0,13.0,...,NaN,NaN,NaN,NaN,ucoin-io/ucoin-python-apiexamples/request_web_...,-1,"""""""\nCopyright 2014-2021 Vincent Texier <vit@...",import asyncio\nimport json\nfrom _socket impo...,"""""""\nCopyright 2014-2021 Vincent Texier <vit@...",1812
26,26,26,helena-project/beetle,controller/gatt/views.py,2,1,15,1,78.0,0.0,...,NaN,NaN,NaN,NaN,helena-project/beetlecontroller/gatt/views.py,-1,"\nfrom django.http import JsonResponse, HttpRe...","from django.http import JsonResponse, HttpResp...","from django.http import JsonResponse, HttpResp...",1852
29,29,29,python-security/pyt,examples/django.nV/taskManager/forms.py,275,166,2145,1,90.0,14.0,...,NaN,NaN,NaN,NaN,python-security/pytexamples/django.nV/taskMana...,-1,# _ _ __ __\n# ...,"from taskManager.models import Project, Task\n...",""""""" forms.py contains various Django forms for...",1959
36,36,36,ray-project/ray,python/ray/experimental/state/common.py,4242,17339,20545,1,94.0,2.0,...,NaN,NaN,NaN,NaN,ray-project/raypython/ray/experimental/state/c...,-1,import logging\n\nfrom dataclasses import data...,import logging\nfrom dataclasses import datacl...,import logging\nfrom dataclasses import datacl...,1643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875012,875012,87848,ManiacalLabs/BiblioPixel,bibliopixel/builder/pixels.py,72,967,249,1,70.0,0.0,...,NaN,NaN,NaN,NaN,ManiacalLabs/BiblioPixelbibliopixel/builder/pi...,-1,"import itertools, weakref\nfrom .. colors impo...","import itertools, weakref\nfrom ..colors impor...","import itertools, weakref\nfrom ..colors impor...",1899
875031,875031,87867,QQuick/Transcrypt,transcrypt/development/automated_tests/re/basi...,231,922,2175,1,38.0,4.0,...,NaN,NaN,NaN,NaN,QQuick/Transcrypttranscrypt/development/automa...,-1,from org.transcrypt.stubs.browser import __pra...,from org.transcrypt.stubs.browser import __pra...,from org.transcrypt.stubs.browser import __pra...,925
875042,875042,87878,softlayer/softlayer-python,SoftLayer/CLI/hardware/authorize_storage.py,180,1060,122,1,25.0,1.0,...,NaN,NaN,NaN,NaN,softlayer/softlayer-pythonSoftLayer/CLI/hardwa...,-1,"""""""Authorize File or Block Storage to a Hardwa...",import click\nimport SoftLayer\nfrom SoftLayer...,"""""""Authorize File or Block Storage to a Hardwa...",812
875071,875071,87907,clic-lab/blocks,BlockWorldRoboticAgent/model/mix_and_gen_q_val...,12,2,39,1,53.0,3.0,...,NaN,NaN,NaN,NaN,clic-lab/blocksBlockWorldRoboticAgent/model/mi...,-1,import tensorflow as tf\n\n\nclass MixAndGener...,import tensorflow as tf\n\nclass MixAndGenerat...,import tensorflow as tf\n\nclass MixAndGenerat...,1881


In [32]:
def undocstring(source):
    try:
        parsed = ast.parse(source)
        return_dict = {'inputs': [], 'labels': []}
        for node in ast.walk(parsed):
            #print("Node is : ",node)
            #print("Node value is : ",node.body[0].value.s)
                    
            if not isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.AsyncFunctionDef)): #, ast.Module
                continue

            if not len(node.body):
                continue

            if not isinstance(node.body[0], ast.Expr):
                continue

            if not hasattr(node.body[0], 'value') or not isinstance(node.body[0].value, ast.Str):
                continue

            # Uncomment lines below if you want print what and where we are removing
            # 
            # 
            return_dict['labels'].append(ast.unparse(node))
            node.body = node.body[1:]
            return_dict['inputs'].append(ast.unparse(node))

        return return_dict
    except:
        parsed = 'nan'
        return parsed

In [33]:
#extract methods from long_df with docstrings from uncommented no docstring input and original method as 
def method_docstring(source):
    methods = undocstring(source)
    inputs = methods['inputs']
    labels = methods['labels']
    #print (methods)
    return inputs, labels
#print(method_docstring(long_df['uncommented_content'][0]))
doc_methods = long_df['uncommented_content'].apply(method_docstring)

In [46]:
#combine short df and long df fixes and save to file
tuple_list = list(doc_methods)
inputs = []
labels = []
for item in tuple_list:
    inputs.extend(item[0])
    labels.extend(item[1])


In [47]:

inputs.extend(list(repaired_df['no_docstring_content']))
labels.extend(list(repaired_df['uncommented_content']))


In [50]:
dict = {'no_docstring_content': inputs, 'uncommented_content': labels}
dataset = Dataset.from_dict(dict).train_test_split(test_size=0.1)

In [51]:
#tokenize and save combined dataset
from transformers import RobertaTokenizer, T5ForConditionalGeneration, PLBartTokenizer

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
example =dataset['train']['uncommented_content']
#label = tokenizer(example, truncation=True)
#print (len(label['input_ids']))
def tokenization(example):
    return_dict = tokenizer(example['no_docstring_content'], padding='max_length', truncation=True)
    #print ((example['content'][10]))
    labels = tokenizer(example['uncommented_content'], padding='max_length', truncation=True).input_ids
    return_dict['labels'] = labels
    #print (return_dict.keys())
    
    return return_dict
train_dataset = dataset["train"].map(tokenization, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
train_dataset.format['type']

test_dataset = dataset["test"].map(tokenization, batched=True)
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.format['type']

loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/vocab.json from cache at /soe/ksmunson/.cache/huggingface/transformers/1e2aacf615bc83f25a9d748eccb762b335eee01a29ab7a8db9b8e86cc851d489.9a48c5abf25554713c6513ab01066e53569b9a2da0d6189715951cf7c6288805
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/merges.txt from cache at /soe/ksmunson/.cache/huggingface/transformers/7eaa9b856402f05e8fdd452951872ecd3c2692ea9abb86b7ab62b07e3bc5f7de.7179059568f1a130b0a79e4bac71f38545207cab0ec45ce82ca09afadb2649a3
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/added_tokens.json from cache at /soe/ksmunson/.cache/huggingface/transformers/a3e93db547e41cdd21f01826d07c5679e111b02d8e969c607611c30a6acbe191.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/special_tokens_map.json from cache at /soe/ksmunson/.cache/huggingface/transformers/5941df5e4315c5ab63

  0%|          | 0/2282 [00:00<?, ?ba/s]

  0%|          | 0/254 [00:00<?, ?ba/s]

'torch'

In [52]:

print ((train_dataset[0]))
train_dataset.save_to_disk(fname_prefix + "datasets/codet5_train_docstring_fixed_bq_padded.hf")
test_dataset.save_to_disk(fname_prefix + "datasets/codet5_test_docstring_fixed_bq_padded.hf")

{'input_ids': tensor([    1,   536,  1002,    67,  1636,    12,  3576,  4672,   203,   565,
         3536,  1477,  1347,  1915,   353,   279,   642,  6199,  8395,   203,
          565,  1002, 12068,    12,  3576,    13,     2,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 